<a href="https://colab.research.google.com/github/gl7176/CNN_tools/blob/main/Points_to_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to our Google Drive folder and pull files
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

If customizing this code, you will need to point the `drive_folder` variable to a URL for your shared google drive folder.

In [42]:
# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1Dav7-r1nxziYVE0ouKaG2xHzr-da9-WF'

!pip install -U -q PyDrive
import os, csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
#    and checks that all necessary files are present


for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  print("Pulled file: " + fname)
  if fname.endswith(".shp"):
    ptfile = fname
  if fname.endswith(".json"):
    tiling_scheme_file = fname

Pulled file: data/Copy of tiling_scheme.json
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp
Pulled file: data/Jbn01_3_03_20_L1A_Seals.cpg
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbn
Pulled file: data/Jbn01_3_03_20_L1A_Seals.dbf
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.prj
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp.xml


In [49]:
!pip install geopandas
!pip install affine

In [54]:
import fiona
with fiona.open(ptfile) as c:
    for i, record in enumerate(c):
        #print(record["geometry"]["coordinates"])
        pt = ogr.Geometry(ogr.wkbPoint) 
        pt.SetPoint_2D(0, record["geometry"]["coordinates"][0], record["geometry"]["coordinates"][1])
        buff = pt.Buffer(1.5)
        box = buff.GetEnvelope()
        print("Envelope: " + str(box))

(435301.04430918256, 2814678.788138528)
(435304.2175890229, 2814677.647183979)
(435308.8170620499, 2814682.674514962)
(435309.6727779618, 2814683.9937436595)
(435304.57268374355, 2814684.9343534363)
(435321.8068295942, 2814679.9019954093)
(435314.7329113884, 2814680.335558138)
(435323.0390605075, 2814685.492672701)
(435339.46332943585, 2814688.548605365)
(435338.16355401324, 2814695.9821524452)
(435327.74977660296, 2814703.853483786)
(435289.1712509039, 2814692.4439382926)
(435264.473874633, 2814699.6535955505)
(435271.24272120267, 2814703.5080776257)
(435268.5477174759, 2814703.288716857)
(435344.97973761125, 2814651.862669075)
(435349.4924530922, 2814646.4235020713)
(435241.6953356113, 2814681.6025100877)
(435230.48826074146, 2814692.566765003)
(435236.49525287165, 2814691.0127172875)
(435241.21716708347, 2814692.925391399)
(435238.43781251577, 2814694.3300114493)
(435244.6540033769, 2814691.4908858156)
(435245.1919429706, 2814690.116151298)
(435237.21250566334, 2814698.1852452043)
(

In [50]:
import json
with open(tiling_scheme_file) as f:
    tiling_scheme = json.load(f)
    proj = tiling_scheme["spatial_reference"]
    gt = tiling_scheme["transform"]

from affine import Affine

geotransform = (gt[2], gt[0], gt[1], gt[5], gt[3], gt[4])
geotransform = Affine.from_gdal(*geotransform)

# define function to transform coordinates from orthomosaic/pixel reference to global reference
def global_transform(box):
    bounding_box = np.array([[box['box'][0], box['box'][1]], [box['box'][2], box['box'][1]], [box['box'][2], box['box'][3]], [box['box'][0], box['box'][3]]]).astype(float)
    count = 0
    for point in bounding_box:
      point = geotransform * point
      bounding_box[count] = point
      count += 1
    box['box'] = bounding_box
    print(box_list[0:3])

In [ ]:
from osgeo import ogr, osr

print(pts[1])

In [3]:
from osgeo import ogr, osr

# EPSG:4326 : WGS84 lat/lon : http://spatialreference.org/ref/epsg/4326/
wgs = osr.SpatialReference()
wgs.ImportFromEPSG(4326)    
coord_trans_cache = {}
def utm_zone(lat, lon):
    """Args for osr.SpatialReference.SetUTM(int zone, int north = 1)"""
    return int(round(((float(lon) - 180)%360)/6)), int(lat > 0)

# Your data from a text file, i.e., fp.readlines()
lines = ['-43.1234,40.1234\n', '-43.1244,40.1244\n']
for line in lines:
    lat, lon = [float(x) for x in line.split(',')]
    # Get projections sorted out for that UTM zone
    cur_utm_zone = utm_zone(lat, lon)
    if cur_utm_zone in coord_trans_cache:
        wgs2utm, utm2wgs = coord_trans_cache[cur_utm_zone]
    else: # define new UTM Zone
        utm = osr.SpatialReference()
        utm.SetUTM(*cur_utm_zone)
        # Define spatial transformations to/from UTM and lat/lon
        wgs2utm = osr.CoordinateTransformation(wgs, utm)
        utm2wgs = osr.CoordinateTransformation(utm, wgs)
        coord_trans_cache[cur_utm_zone] = wgs2utm, utm2wgs
    # Create 2D point
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.SetPoint_2D(0, lon, lat) # X, Y; in that order!
    orig_wkt = pt.ExportToWkt()
    # Project to UTM
    res = pt.Transform(wgs2utm)
    if res != 0:
        print("spatial transform failed with code " + str(res))
    print(orig_wkt + " -> " + pt.ExportToWkt())
    # Compute a 15 m buffer
    buff = pt.Buffer(15)
    # Transform UTM buffer back to lat/long
    res = buff.Transform(utm2wgs)
    if res != 0:
        print("spatial transform failed with code " + str(res))
    print("Envelope: " + str(buff.GetEnvelope()))
    # print("WKT: " + buff.ExportToWkt())

POINT (40.1234 -43.1234) -> POINT (591383.421820465 5224869.25719953)
Area: 0.0007065354092777124 km^2
Envelope: (40.123215635834605, 40.12358436414829, -43.12353504742207, -43.1232649525746)
POINT (40.1244 -43.1244) -> POINT (591463.277979514 5224757.11415603)
Area: 0.0007065354092740745 km^2
Envelope: (40.12421563289496, 40.12458436708794, -43.12453504735205, -43.12426495264462)
